## 問題

https://twitter.com/e869120/status/1381376542836596737

## 解説

https://twitter.com/e869120/status/1381739128291614720

In [3]:
#r "nuget: FSharpPlus"

Installed Packages FSharpPlus, 1.3.2

In [4]:
open System
open System.Numerics
open System.Collections.Generic
open FSharpPlus

In [5]:
// Union-Findを使って判定したいので、Union-Find木を実装

type [<Struct>] UnionFindTreeId = UnionFindTreeId of int
with
  static member op_Implicit (x: int) = UnionFindTreeId x

type UnionFindTree(count: int) =
  let parents = Array.create count -1
  member _.count = parents.Length
  member this.findRoot((UnionFindTreeId i) as treeId) =
    if parents[i] < 0 then treeId else
    let (UnionFindTreeId root) as rootId = this.findRoot(parents[i])
    parents[i] <- root
    rootId
  member this.size x =
    let (UnionFindTreeId root) = this.findRoot x
    -parents[root]
  member this.isSame x y = this.findRoot(x) = this.findRoot(y)
  member this.union x y =
    let x, y = this.findRoot x, this.findRoot y
    if x = y then false else
    let (UnionFindTreeId small), (UnionFindTreeId large) = if this.size x < this.size y then x, y else y, x
    parents[large] <- parents[large] + parents[small]
    parents[small] <- large
    true

In [6]:
let solve H W Qs =
  // 元の問題だと直感に反してH(height)がx、W(width)がyなのでそれに合わせる
  // フィールドの2次元配列は上下左右に余分に1マス分確保して、絶対赤くならない番兵とする
  let field = Array2D.zeroCreate<bool> (H + 2) (W + 2)
  let uft = UnionFindTree(H * W)
  let getId (x, y) = (x - 1) * W + (y - 1) |> UnionFindTreeId
  Qs |> Seq.choose (function
    | [1; x; y] when not field[x, y] ->
      // 自身のマスを赤くし、上下左右の赤マスと結合する
      field[x, y] <- true
      let uftId = getId (x, y)
      let redCells = [x - 1, y; x + 1, y; x, y - 1; x, y + 1] |> filter (fun (x, y) -> field[x, y])
      redCells |> List.iter (getId >> uft.union uftId >> ignore)
      None
    | [1; _; _] -> None
    | [2; xa; ya; xb; yb] ->
      // 始点と終点がそもそも赤いのが前提条件、その上で始点と終点が結合されているかどうか
      let result = field[xa, ya] && field[xb, yb] && uft.isSame (getId (xa, ya)) (getId (xb, yb))
      Some result
    | otherwise -> failwith $"{otherwise} は入力として不正")
  |> Seq.toList
  

In [7]:
solve 3 3 [
  [1; 2; 2]
  [1; 1; 1]
  [2; 1; 1; 2; 2]
  [1; 3; 2]
  [2; 1; 1; 2; 2]
  [2; 2; 2; 3; 2]
  [1; 2; 3]
  [1; 2; 1]
  [2; 1; 1; 2; 2]
  [2; 1; 1; 3; 3]
]

index,value
0,False
1,False
2,True
3,True
4,False


In [8]:
solve 1 1 [
  [2; 1; 1; 1; 1]
  [1; 1; 1]
  [2; 1; 1; 1; 1]
]

index,value
0,False
1,True
